In [838]:
import requests as r
import pandas as pd 
from bs4 import BeautifulSoup as bs
import re
import time as t
from requests import Session
import json
import datetime

In [982]:
def request(url, pages=1): 
    
    url = 'https://www.google.com/search?q=bitcoin&client=firefox-b-1-d&tbm=nws&ei=jWIWXYnSDorZ5gLo8JbIDg&start='+str(pages)+'&sa=N&ved=0ahUKEwjJzJ6K7YzjAhWKrFkKHWi4Bek4WhDy0wMIXA&biw=1320&bih=737&dpr=1.09'
    soup_string_list = []
    soup_list = []
    if pages != 1:
        
        try:
            
            for i in range(pages):

                pages += (i*10)-10
                html = r.get(url, timeout=1).content
                soup = bs(html, 'lxml')

                soup_string = str(bs(html, 'lxml'))
                soup_string_list.append(soup_string)
                soup_string = ''.join(soup_string_list)

                soup_list.append(soup)
                
                t.sleep(1)
            return soup_string, soup_list
        except: 
            pass
        
    else:
        html = r.get(url, timeout=1).content
        soup_string = str(bs(html, 'lxml'))
        
        soup = bs(html, 'html.parser')
        soup.prettify()
        
        return soup_string, soup
    
    
    
def clean_titles(soup_string):
    
    search = re.findall('">.{0,200}</a></h3>', soup_string)
    sub_1 = list(map(lambda x: re.sub(r"(</b>)",r"",x), search))
    sub_2 = list(map(lambda x: re.sub(r"(<b>)",r"",x), sub_1))
    rstrip = list(map(lambda x: x.strip('</a></h3>'), sub_2))
    lstrip = list(map(lambda x: x.lstrip('">'), rstrip))
    
    return lstrip


def clean_sources(soup): 
    clean = []
    
    for i in soup:
        search = i.find_all('div', attrs={'class':'slp'})
        text = [i.text for i in search]
        split = list(map(lambda x: x.split('-'), text))
        rstrip = list(map(lambda x: x[0].rstrip(), split))
        
        clean.append(rstrip)
        
    flatten = [i for mid in clean for i in mid]
    
    return flatten



def clean_dates(soup): 
    clean = []
    
    for i in soup: 
        search = i.find_all('div', attrs={'class':'slp'})
        text = [i.text for i in search]
        split = list(map(lambda x: x.split('-'), text))
        rstrip = list(map(lambda x: x[1].rstrip(), split))
        lstrip = list(map(lambda x : x.lstrip(), rstrip))
        
        clean.append(lstrip)
        
    flatten = [i for mid in clean for i in mid]
    
    return flatten



def format_date(series=df_daily['Date']):

    unformated = list(series)
    formated = []
    try: 
        for i in unformated: 
            if '2019' in i:  

                take = datetime.datetime.strptime(i, '%b %d, %Y')
                cut = take.isoformat()[:10]
                formated.append(cut)
    except:
        formated.append(i)

    return formated



def BTC(start='2019-01-01', end='2019-07-25'):  
    
    key = 'fcf1f334-f9b2-4acd-8024-3b425c1272a4'
    domain = 'https://api.coindesk.com/v1/bpi/historical/close.json?start=%s&end=%s'%(start,end)
    JSON = r.get(domain).json()['bpi']
    keys = list(JSON)
    
    df = pd.DataFrame()
    df['Date'] = keys
    
    values = []
    for i in keys: 
        values.append(JSON[i])
        
    df['Price'] = values
    
    return df

In [987]:
%%time
titles, DatesAndSources = request(url,150)

TypeError: cannot unpack non-iterable NoneType object

In [988]:
print('Titles Length:%s\nSources Length:%s\nDates Length:%s'%(len(clean_titles(titles)),
                                                              len(clean_sources(DatesAndSources)),
                                                             len(clean_dates(DatesAndSources))))

Titles Length:0
Sources Length:0
Dates Length:0


In [985]:
df = pd.DataFrame()

In [986]:
df['Title'] = clean_titles(titles)
df['Source'] = clean_sources(DatesAndSources)
df['Date'] = clean_dates(DatesAndSources)
df[df['Date']=='3 days ago']
df.head()

/anaconda3/lib/python3.7/site-packages/pandas/core/ops.py:1167: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


TypeError: invalid type comparison

In [ ]:
df_daily.shape

In [ ]:
df_daily = df[df['Date'].str.contains('2019|2018')]

In [ ]:
print(len(df_daily['Date']))
print(len(format_date()))

In [ ]:
df_daily['Date'] = format_date()

In [ ]:
df_daily.head()

In [ ]:
mini = df_daily['Date'].min()
maxi = df_daily['Date'].max()


In [962]:
join_df = BTC(mini, maxi).join(df_daily.set_index('Date'),on='Date')

In [964]:
join_df.shape

(332, 4)

In [979]:
unique_df = join_df.drop_duplicates('Title').reset_index()

In [981]:
unique_df

,index,Date,Price,Title,Source
0,0,2019-05-30,8273.0117,How The Super-Rich Are Buying Up Bitcoin,Forbes
1,1,2019-05-31,8544.9700,NaN,NaN
2,10,2019-06-09,7634.6483,A Bitcoin 'Millionaire' Is Planning To 'Resurr...,Forbes
3,19,2019-06-18,9083.8167,'Bitcoin is easily going to take out its all-t...,CNBC
4,20,2019-06-19,9288.5367,'Hard Core Fund' Collects 50 BTC to Support Bi...,CoinDesk
5,21,2019-06-20,9539.7883,Bitcoin Price Breaches $9.6K to Hit 400-Day Hig,CoinDesk
6,22,2019-06-21,10232.9167,What are the options for an Indian investor if...,Quartz


pandas.core.frame.DataFrame